In [75]:
import trusspy as tp
import numpy as np
import pandas as pd
import random


### Materiais

In [76]:
Young_modulus = [  # módulo de elasticidade
    200e9,
]

fy_k = [  # tensão de escoamento
    344.7e6,  # ASTM A572 G50
    413.6e6,  # ASTM A572 G60
]

density = [
    7836.41 # kg/m**3
]

materials = np.array([
    [Young_modulus[0], fy_k[0], density[0]],
    [Young_modulus[0], fy_k[1], density[0]],
])

### Seções

In [77]:
secoes = np.array([
    [2.35E-04],  # 1 L40x40x3
    [3.08E-04],  # 2 L40x40x4
    [3.79E-04],  # 3 L40x40x5
    [2.66E-04],  # 4 L45x45x3
    [3.49E-04],  # 5 L45x45x4
    [4.30E-04],  # 6 L45x45x5
    [2.96E-04],  # 7 L50x50x3
    [3.89E-04],  # 8 L50x50x4
    [4.80E-04],  # 9 L50x50x5
    [4.71E-04],  # 10 L60x60x4
    [5.82E-04],  # 11 L60x60x5
    [5.13E-04],  # 12 L65x65x4
    [6.31E-04],  # 13 L65x65x5
])

##### Creating DataFrame

In [78]:
data = pd.DataFrame(columns=("Fx", "Fy","area_secao", "deformação_x", "deformação_y"))


In [79]:
class trelica:

    def __init__(self, fx=0, fy=-10**5, E=200*10**9, area=10**-4):
        self.fx = fx
        self.fy = fy
        self.E = E
        self.area = area        
        

    def deformacao(self):

        element_type = 1    # truss
        material_type = 1    # linear-elastic

        L1 = 1
        L2 = 1
        L3 = 1

        E = self.E

        area = self.area

        F1 = self.fx
        F2 = self.fy
        F3 = 0
        
        M = tp.Model()

        with M.Nodes as MN:
            MN.add_node(1, coord=(0,    0,  0))
            MN.add_node(2, coord=(0,    0,  L1))
            MN.add_node(3, coord=(L2,   0,  0))
            MN.add_node(4, coord=(L2,   0,  L1))
            MN.add_node(5, coord=(L2+L3, 0,  0))

        with M.Elements as ME:
            ME.add_element(1, conn=(1, 3), gprop=[area])
            ME.add_element(2, conn=(1, 4), gprop=[area])
            ME.add_element(3, conn=(2, 3), gprop=[area])
            ME.add_element(4, conn=(2, 4), gprop=[area])
            ME.add_element(5, conn=(3, 4), gprop=[area])
            ME.add_element(6, conn=(3, 5), gprop=[area])
            ME.add_element(7, conn=(4, 5), gprop=[area])

            ME.assign_etype('all',   element_type)
            ME.assign_mtype('all',  material_type)
            ME.assign_material('all', [E])

        with M.Boundaries as MB:
            MB.add_bound_U(1, (0, 0, 0))
            MB.add_bound_U(2, (0, 0, 0))
            MB.add_bound_U(3, (1, 0, 1))
            MB.add_bound_U(4, (1, 0, 1))
            MB.add_bound_U(5, (1, 0, 1))

        with M.ExtForces as MF:
            MF.add_force(5, (F1, F3, F2))

        # M.Settings.dlpf = 1
        # M.Settings.du = 1

        M.Settings.incs = 1

        # M.Settings.stepcontrol = True
        # M.Settings.maxfac = 1

        # M.Settings.ftol = 10**8
        # M.Settings.xtol = 1
        # M.Settings.nfev = 1

        # M.Settings.dxtol = 1

        # build model and run job
        M.build()
        M.run()
        
        dx = M.Results.R[-1].U[-1][0]
        dy = M.Results.R[-1].U[-2][-1]
                
        return dx, dy

        # M.plot_history(nodes=[5,5], X='Displacement X', Y='Displacement Z')

        # M.plot_model(config=['undeformed'],
        #                        view='xz', #'xy', 'yz', 'xz'
        #                        contour='force',
        #                     #    lim_scale=(-3,2,0,5,-1,4), #3d
        #                        lim_scale=1.4, #plane-view
        #                        force_scale=2.0, #5
        #                        inc=0)

        # M.plot_model(config=['deformed'],
        #                        view='xz',
        #                        contour='force',
        #                        lim_scale=1.3,
        #                        force_scale=500.0,
        #                        inc=-1)

        # M.plot_model(config=['deformed'],
        #                        view='xz',
        #                        contour='force',
        #                     #    lim_scale=(-3,2,0,5,-2,3),
        #                         lim_scale=1.3,
        #                        force_scale=500,
        #                        inc=-1)


### Generate data

In [80]:
for _ in range(100):
    fx = random.randint(-10**5, 10**5)  # em Newtons
    fy = random.randint(-10**5, 10**5)  # em Newtons
    E = materials[random.randint(0, materials.shape[0]-1)][0] # E
    area = secoes[random.randint(0, secoes.shape[0]-1)][0] # Area
    
    row = trelica(fx, fy, E, area)
    dx, dy = row.deformacao()
    data.loc[len(data)] = [fx, fy, area, dx, dy]



 _____                  ______      
|_   _|                 | ___ \     
  | |_ __ _   _ ___ ___ | |_/ /   _ 
  | | '__| | | / __/ __||  __/ | | |
  | | |  | |_| \__ \__ \| |  | |_| |
  \_/_|   \__,_|___/___/\_|   \__, |
                               __/ |
                              |___/ 

TrussPy - Truss Solver for Python
          Version 1.0.3

Dutzler Andreas, Graz University of Technology, 2023
        

# Initialize Model
* loading Managers

    - finished.


# Model Summary
    Analysis Dimension      "ndim": 3
    Number of Nodes       "nnodes": 5
    Number of Elements    "nelems": 7
 
    System DOF              "ndof": 15
    active DOF             "ndof1": 6
    locked DOF             "ndof2": 9
 
    active DOF          "nproDOF1": [ 6  8  9 11 12 14]
    fixed  DOF          "nproDOF0": [ 0  1  2  3  4  5  7 10 13]
\pagebreak
 
# Run Simulation

## Summary of Analysis Parameters
|Description                          |Parameter|Value|
|:------------------------------

In [81]:
data.tail()


,Fx,Fy,area_secao,deformação_x,deformação_y
95,-77962.0,-66521.0,0.000296,-0.000105,-0.000082
96,92016.0,39966.0,0.000349,0.000078,0.000048
97,2211.0,99901.0,0.000430,0.000058,0.000073
98,58931.0,22544.0,0.000349,0.000048,0.000028
99,18398.0,90356.0,0.000582,0.000044,0.000050


In [82]:
# M.plot_model(config=['deformed'],
#              view='xz',
#              contour='force',
#              lim_scale=(-0.5, 3.5, -2, 2),
#              force_scale=1/2000,
#              inc=-1)

# M.plot_history(nodes=[5, 5], X='Displacement Z', Y='Force Z')

# # show results
# M.plot_movie(config=['deformed'],
#              view='xz',
#              contour='force',
#              lim_scale=(-0.5, 3.5, -2, 2),
#              force_scale=5,
#              cbar_limits=[-1, 1])

In [84]:
data.to_excel("tabela.xlsx", index=False)